# Gen2D

This notebook shows a simple model for clustering a 2D image into different components.

In [ ]:
import jax
import jax.numpy as jnp
import model
from scipy import misc

from genjax import ChoiceMapBuilder as C
from genjax import pretty

pretty()

## Model

### Testing to sample from model

In [ ]:
image = misc.face()
# plt.imshow(image)
# plt.show()

H, W, _ = image.shape

image.shape
hypers = model.Hyperparams(
    a_x=1.0,
    b_x=1.0,
    a_y=1.0,
    b_y=1.0,
    mu_x=0.0,
    mu_y=0.0,
    a_rgb=jnp.array([1.0, 1.0, 1.0]),
    b_rgb=jnp.array([1.0, 1.0, 1.0]),
    alpha=1.0,
    sigma_xy=jnp.array([1.0, 1.0]),
    sigma_rgb=jnp.array([1.0, 1.0, 1.0]),
    n_blobs=100,
    H=H,
    W=W,
)

key = jax.random.PRNGKey(0)
key, subkey = jax.random.split(key)
tr = model.model.simulate(subkey, (hypers,))

### Image processing

In [ ]:
flattened_image = jnp.concatenate(
    (jnp.indices((H, W)).reshape(H * W, 2), image.reshape(H * W, 3)), axis=1
)

xy, rgb = flattened_image[:, :2], flattened_image[:, 2:]

### Initial trace for inference

In [ ]:
key, subkey = jax.random.split(key)
obs = C["likelihood_model", "xy"].set() ^ C["likelihood_model", "rgb"].set()

tr = model.model.importance(subkey, obs, hypers)

## Inference